### Extracao e Limpeza

In [1]:
import pandas as pd
from os import listdir, path

In [3]:
directory = 'data_raw'
cols = ['ANO', 'MES', 'EMPRESA', 'ORIGEM', 'DESTINO', 'TARIFA', 'ASSENTOS']

In [4]:
dfs = []
for filename in listdir(directory):
    f = path.join(directory, filename)
    dfs.append(pd.read_csv(str(f), sep=';', encoding='latin-1', names=cols, skiprows=1, low_memory=False))

df = pd.concat(dfs)

In [5]:
df['TARIFA'] = df['TARIFA'].str.replace(',', '.')
df['TARIFA'] = df['TARIFA'].astype(float)

In [6]:
df_aerodromos = pd.read_csv('AerodromosPublicos.csv', sep=';', encoding='latin-1', low_memory=False)

In [7]:
drop_cols = ['CIAD', 'Nome', 'Município Servido', 'UF', 'UF Servido', 'LATGEOPOINT', 'LONGEOPOINT', 'Latitude', 'Longitude', 
             'Altitude', 'Operação Diurna', 'Operação Noturna', 'Designação 1', 'Superfície 1', 'Superfície 1', 'Designação 2',
            'Resistência 1', 'Resistência 2', 'Superfície 2', 'Situação', 'Validade do Registro', 'Portaria de Registro', 'Link Portaria',
            'Largura 1', 'Largura 2', 'Comprimento 1', 'Comprimento 2']

In [8]:
df_aerodromos.drop(drop_cols, axis=1, inplace=True)
df_aerodromos.rename(columns={'Código OACI': 'oaci', 'Município': 'cidade'}, inplace=True)
df_aerodromos.set_index('oaci', inplace=True)

### Join do DF principal com o DF de aerodromos

In [9]:
df = df.merge(df_aerodromos, left_on='ORIGEM', right_on='oaci')
df = df.merge(df_aerodromos, left_on='DESTINO', right_on='oaci')

In [10]:
df.drop(['ORIGEM', 'DESTINO'], axis=1, inplace=True)
df.rename(columns={'cidade_x': 'ORIGEM', 'cidade_y': 'DESTINO'}, inplace=True)

In [11]:
df.to_csv(path.join('data_transformed', 'df_main.csv'), index=False)